In [2]:
import numpy as np
from numpy import linalg as la

In [3]:
from numpy import *
U,Sigma,VT=np.linalg.svd([[1, 1],[7, 7]])

In [4]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [5]:
Sigma

array([1.00000000e+01, 2.82797782e-16])

In [6]:
VT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [7]:
def loadExData():
    return[[1, 1, 1, 0, 0],
            [2, 2, 2, 0, 0],
            [1, 1, 1, 0, 0],
            [5, 5, 5, 0, 0],
            [1, 1, 0, 2, 2],
            [0, 0, 0, 3, 3],
            [0, 0, 0, 1, 1]]

In [10]:
Data=loadExData()
U,Sigma,VT=linalg.svd(Data)
Sigma

array([9.72140007e+00, 5.29397912e+00, 6.84226362e-01, 4.11502614e-16,
       1.36030206e-16])

In [11]:
Sig3=mat([[Sigma[0], 0, 0],[0, Sigma[1], 0], [0, 0, Sigma[2]]])
U[:,:3]*Sig3*VT[:3,:]

matrix([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          7.75989921e-16,  7.71587483e-16],
        [ 2.00000000e+00,  2.00000000e+00,  2.00000000e+00,
          3.00514919e-16,  2.77832253e-16],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          2.18975112e-16,  2.07633779e-16],
        [ 5.00000000e+00,  5.00000000e+00,  5.00000000e+00,
          3.00675663e-17, -1.28697294e-17],
        [ 1.00000000e+00,  1.00000000e+00, -5.48397422e-16,
          2.00000000e+00,  2.00000000e+00],
        [ 3.21319929e-16,  4.43562065e-16, -3.48967188e-16,
          3.00000000e+00,  3.00000000e+00],
        [ 9.71445147e-17,  1.45716772e-16, -1.52655666e-16,
          1.00000000e+00,  1.00000000e+00]])

In [29]:
# 欧式距离函数 -> 相似度
def ecludSim(inA,inB):
    # la.form求A和B的欧式距离
    # 距离与相似度程相反的关系，因此取倒数
    # 为防止分母为0，求得的距离再加1
    # 计算结果为(0, 1]
    # 缺点：一个特征不一样就会导致距离很大
    print(inA, inB)
    return 1.0/(1.0 + la.norm(inA - inB))

# pearson相似度
def pearsSim(inA,inB):
    # 判断两组数据与某一直线的拟合程度
    # 优点：数据不规范时效果好
    if len(inA) < 3 : return 1.0
    # 结果为[-1,1]，需要转到[0,1]区间
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

# 余弦相似度：只考虑两组数据之间的夹角
def cosSim(inA,inB):
    # 不会因为文章的长度不同导致结果偏差太大
    # 结果为[-1,1]，需要转到[0,1]区间
    num = float(inA.T.dot(inB))
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [30]:
myMat=np.array(loadExData())
ecludSim(myMat[:,0],myMat[:,4])  # 三种距离的计算结果都和书上不一样，难道是用的数据不同？

[1 2 1 5 1 0 0] [0 0 0 0 2 3 1]


0.13367660240019172

In [22]:
ecludSim(myMat[:,0],myMat[:,0])

1.0

In [23]:
cosSim(myMat[:,0],myMat[:,4])

0.5472455591261534

In [25]:
cosSim(myMat[:,0],myMat[:,0])

0.9999999999999999

In [26]:
pearsSim(myMat[:,0],myMat[:,4])

0.23768619407595815

In [28]:
pearsSim(myMat[:,0],myMat[:,0])

1.0

In [31]:
def standEst(dataMat, user, simMeas, item):
    ratedUser = dataMat[:, dataMat[:,item]!=0]
    totalSim, totalScore = 0
    for userToCompare in ratedUser:
        sim = simMeas(userToCompare, user)
        totalSim += sim
        totalScore += sim * dataMat[userToCompare, item]
    return totalScore/totaSim
    
# dataMat:一行代表一个User，一列代表一个菜
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    # 1 Look for things the user hasn’t yet rated: look for values with 0 in the user-item matrix.
    notRated = dataMat[:, dataMat[user,:]==0]
    # 2 Of all the items this user hasn’t yet rated, find a projected rating for each item:
    # that is, what score do we think the user will give to this item? 
    notRatedScorePredict = []
    for dish in notRated:
        notRatedScorePredict.append(standEst(dataMat, user, simMeas, dish))
    # 3 Sort the list in descending order and return the first N items.
    notRatedScorePredict.sort(reverse=True)
    return notRatedScorePredict

In [33]:
myMat=np.array(loadExData())
myMat[0,1]=myMat[0,0]=myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
myMat

array([[4, 4, 1, 0, 0],
       [4, 2, 2, 0, 0],
       [4, 1, 1, 0, 0],
       [5, 5, 5, 2, 0],
       [1, 1, 0, 2, 2],
       [0, 0, 0, 3, 3],
       [0, 0, 0, 1, 1]])